In [2]:
# prompt: short boundry detection and background blured using web cam

import cv2
import numpy as np

def blur_background(frame, threshold=200):
    """
    Performs short-boundary detection and background blurring.

    Args:
      frame: The input video frame.
      threshold: The threshold for detecting edges.

    Returns:
      The frame with the background blurred.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, threshold)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
      largest_contour = max(contours, key=cv2.contourArea)
      mask = np.zeros_like(gray)
      cv2.drawContours(mask, [largest_contour], -1, (255), thickness=cv2.FILLED)
      
      # Blur the background
      background = cv2.GaussianBlur(frame, (51, 51), 0)  # Increase kernel size for more blur
      
      # Combine the blurred background with the foreground
      foreground = cv2.bitwise_and(frame, frame, mask=mask)
      blurred_background = cv2.bitwise_and(background, background, mask=cv2.bitwise_not(mask))
      result = cv2.add(foreground, blurred_background)
      return result
    else:
      return frame  # Return original frame if no contours found


# Access the webcam
cap = cv2.VideoCapture(0)  # 0 usually refers to the default webcam

if not cap.isOpened():
  print("Error opening video capture")
  exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background blurring
    processed_frame = blur_background(frame)
    
    cv2.imshow('Background Blur', processed_frame)


    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()